### Install langchain pacakage

In [94]:
!pip install --quiet langchain langchain-community langchainhub langchain-openai langchain-chroma bs4 datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


### save key for conveinience

In [54]:
from google.colab import userdata

### Langchain support loading data from multiple source
1. crawl webpage content
<font color="red"> explain chunk overlap here </font>

In [111]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
# https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html
# default parser: bs4, parse webcontent (basically remove html tags)
loader = WebBaseLoader(web_path="https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html",
                       bs_kwargs = dict(parse_only=bs4.SoupStrainer(
                          class_=("py method")
                       )))
# TODO: explain what is "document" here,
# ref: https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document
docs = loader.load()

In [42]:
print(vars(docs[0]).keys())
print(docs[0].page_content)

dict_keys(['page_content', 'metadata', 'type'])


__init__(web_path: Union[str, Sequence[str]] = '', header_template: Optional[dict] = None, verify_ssl: bool = True, proxies: Optional[dict] = None, continue_on_failure: bool = False, autoset_encoding: bool = True, encoding: Optional[str] = None, web_paths: Sequence[str] = (), requests_per_second: int = 2, default_parser: str = 'html.parser', requests_kwargs: Optional[Dict[str, Any]] = None, raise_for_status: bool = False, bs_get_text_kwargs: Optional[Dict[str, Any]] = None, bs_kwargs: Optional[Dict[str, Any]] = None, session: Optional[Any] = None) → None[source]Â¶
Initialize loader.

Parameters

web_paths (Sequence[str]) â€“ Web paths to load from.
requests_per_second (int) â€“ Max number of concurrent requests to make.
default_parser (str) â€“ Default parser to use for BeautifulSoup.
requests_kwargs (Optional[Dict[str, Any]]) â€“ kwargs for requests
raise_for_status (bool) â€“ Raise an exception if http status code denotes an error.
bs

In [43]:
# split long text
# all of the text splitter are listed here: https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=512,
    add_start_index=True, # TODO: explain parameter, ref: https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html
    separators=["\n\n\n"]
)
split_docs = text_splitter.split_documents(docs)
print(len(split_docs))

4


In [44]:
for doc in split_docs:
  print('*'*10, '\n', doc.page_content)

********** 
 

__init__(web_path: Union[str, Sequence[str]] = '', header_template: Optional[dict] = None, verify_ssl: bool = True, proxies: Optional[dict] = None, continue_on_failure: bool = False, autoset_encoding: bool = True, encoding: Optional[str] = None, web_paths: Sequence[str] = (), requests_per_second: int = 2, default_parser: str = 'html.parser', requests_kwargs: Optional[Dict[str, Any]] = None, raise_for_status: bool = False, bs_get_text_kwargs: Optional[Dict[str, Any]] = None, bs_kwargs: Optional[Dict[str, Any]] = None, session: Optional[Any] = None) → None[source]Â¶
Initialize loader.

Parameters

web_paths (Sequence[str]) â€“ Web paths to load from.
requests_per_second (int) â€“ Max number of concurrent requests to make.
default_parser (str) â€“ Default parser to use for BeautifulSoup.
requests_kwargs (Optional[Dict[str, Any]]) â€“ kwargs for requests
raise_for_status (bool) â€“ Raise an exception if http status code denotes an error.
bs_get_text_kwargs (Optional[Dict[str

2. load HuggingFace dataset

In [67]:
import datasets
from datasets import load_dataset

dataset = load_dataset("MediaTek-Research/TCEval-v2", 'drcd')
clean_test = dataset['test'].to_pandas()
clean_dev = dataset['dev'].to_pandas()
clean_test = clean_test[:1000].copy(deep=True).reset_index(drop=True)
len(clean_test)

1000

In [68]:
# choose queston that only has one answer
clean_test = clean_test[clean_test['references'].apply(len)==1]
clean_dev = clean_dev[clean_dev['references'].apply(len)==1]
clean_test['references'] = clean_test['references'].apply(lambda x: x[0])
clean_dev['references'] = clean_dev['references'].apply(lambda x: x[0])
clean_test = clean_test.rename(columns={"paragraph":"context", "references":"answer"})
clean_dev = clean_dev.rename(columns={"paragraph":"context", "references":"answer"})

<ipython-input-68-0c24d654ec0b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_test['references'] = clean_test['references'].apply(lambda x: x[0])
<ipython-input-68-0c24d654ec0b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_dev['references'] = clean_dev['references'].apply(lambda x: x[0])


In [69]:
# build documents as from raw text
docs = []
from langchain.docstore.document import Document
for _, record in clean_test.iterrows():
    docs.append(Document(page_content=record['context'], metadata={"id": record['id']}))

## Embedding Model






In [48]:
!pip install -q langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 62.7 MB/s eta 0:00:00


1. HuggingFace embedding model [API document](dochttps://api.python.langchain.com/en/latest/embeddings/langchain_huggingface.embeddings.huggingface.HuggingFaceEmbeddings.html)

In [49]:
# This packs HF-model as a function
# TODO: explain embeddin gmodel, ref: https://api.python.langchain.com/en/latest/embeddings/langchain_core.embeddings.embeddings.Embeddings.html#langchain_core.embeddings.embeddings.Embeddings
from langchain_huggingface import HuggingFaceEmbeddings
embed_func = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [51]:
embedded_sentence = embed_func.embed_query("This is a test sentence.")
print(len(embedded_sentence))

384


In [52]:
embedded_documents = embed_func.embed_documents(["This is a test sentence.", "This is another test sentence."])
print(len(embedded_documents))
print(len(embedded_documents[0]))

2
384


2. OpenAI embedding

In [65]:
from langchain_openai import AzureOpenAIEmbeddings


embed_func = AzureOpenAIEmbeddings(deployment="text-embedding-3-large",
                               openai_api_key=userdata.get('mtk-open-key'),
                               azure_endpoint='https://mtknycuragwebinar.openai.azure.com/',
                               openai_api_version='2024-05-01-preview',
                               max_retries=10,
                               chunk_size=128,
                               show_progress_bar=True)

In [58]:
embedded_sentence = embed_func.embed_query("This is a test sentence.")
print(len(embedded_sentence))

  0%|          | 0/1 [00:00<?, ?it/s]

3072


[Embedding model Leaderboard](https://huggingface.co/spaces/mteb/leaderboard)

3. (advanced) Custom embedding model

In [63]:
from langchain_core.embeddings import Embeddings
from typing import List
class CustomEmbeddings(Embeddings):
    def __init__(self):
        pass
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [[0, 0, 0] for i in range(len(texts))]

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

embed_func = CustomEmbeddings()

In [64]:
embed_func.embed_query('This is a test sentence')

[0, 0, 0]

## vector store
- create database

In [66]:
print(docs)

[Document(page_content="\n\n__init__(web_path: Union[str, Sequence[str]] = '', header_template: Optional[dict] = None, verify_ssl: bool = True, proxies: Optional[dict] = None, continue_on_failure: bool = False, autoset_encoding: bool = True, encoding: Optional[str] = None, web_paths: Sequence[str] = (), requests_per_second: int = 2, default_parser: str = 'html.parser', requests_kwargs: Optional[Dict[str, Any]] = None, raise_for_status: bool = False, bs_get_text_kwargs: Optional[Dict[str, Any]] = None, bs_kwargs: Optional[Dict[str, Any]] = None, session: Optional[Any] = None) → None[source]Â¶\nInitialize loader.\n\nParameters\n\nweb_paths (Sequence[str]) â€“ Web paths to load from.\nrequests_per_second (int) â€“ Max number of concurrent requests to make.\ndefault_parser (str) â€“ Default parser to use for BeautifulSoup.\nrequests_kwargs (Optional[Dict[str, Any]]) â€“ kwargs for requests\nraise_for_status (bool) â€“ Raise an exception if http status code denotes an error.\nbs_get_text_kw

In [75]:
# store vectors to chroma db
# chroma API doc: https://python.langchain.com/v0.1/docs/integrations/vectorstores/chroma/
from langchain_chroma import Chroma

# approach 1: from langchain documents
vectorstore = Chroma.from_documents(documents=docs, embedding=embed_func)
print(len(vectorstore.get()['documents']))

  0%|          | 0/7 [00:00<?, ?it/s]

800


In [ ]:
# TODO: chroma attribute, explain process
# delete previous embeddings
# Chroma().delete_collection()

- query database

In [76]:
# approach 1: directly use vectorstore to query
q="決勝盤會在哪一種網球比賽中使用？"
vectorstore.similarity_search_with_score(q, k=5)

  0%|          | 0/1 [00:00<?, ?it/s]

[(Document(page_content='大多數比賽由多盤組成，而且總盤數一般為單數。大滿貫男子單打比賽有5盤，率先贏得3盤的選手獲勝；而其他比賽一般有3盤，率先贏得2盤的選手獲勝。發球權在每局比賽後交換，不受一盤比賽的開始或結束影響。決勝局亦算作一局比賽。一場比賽的總比分可以只給出總盤數的比分，也可列出所有局數的比分，但必須先列出勝者的比分。例如7-56-76-47-6採用的就是後一種記分方式，表示獲勝的一方贏得了第一、第三和第四盤的勝利，而第二和第四盤是靠決勝局決出勝負的。在括號中的數字表示的是輸的一方在決勝局中得到的分數，例如第二盤決勝局的比分就是4-7，而第四盤決勝局的比分則是8-6。在現在網球比賽中，為了加快比賽進程，在雙方平局局數足夠高時將不再延長比賽而採用一局特殊的決勝局結束一盤比賽。決勝局規則為現在大多數公開賽所採用，但並不是強制使用，如溫布頓網球錦標賽在第五盤時仍然採用長盤勝負。在一般的決勝局中，首先獲得7分且淨勝2分以上的選手贏得該盤比賽。由於在決勝局中雙方交替發球，無論哪一方獲勝該局均不會被記入保發/破發局。在雙打比賽中，有時也會使用一種特殊的決勝盤。如果雙方在前兩盤各下一盤，那麼第三盤將為一局決勝負，首先贏得10分且淨勝2分以上的一方贏得比賽。', metadata={'id': 'test-33'}),
  0.8968632817268372),
 (Document(page_content='網球正拍擊球指右手球員從身體右側擊球，擊球後球拍從體前划過，最後停留在身體左側。左手球員相反。它是網球基本技術中常用的擊球方法，往往是初學者最先學習的技術，包括上旋球、下旋球、平擊球等。正手上旋球速度快，而且不易失誤，往往用於進攻和雙方僵持對拉。正手的握拍有大陸式、東方式、半西方式和西方式。大陸式可以打出下旋球，同時方便反手擊球。半西方式和西方式有利於打出強烈的上旋。正手一般是單手握拍，但也有一些優秀運動員採用正手雙手握拍。反手擊球指右手球員從身體左側引拍擊球，擊球後球拍從體前划過，最後停留在身體右側。左手球員相反。相對於正手擊球，反手擊球較難掌握。反手擊球根據球的旋轉可分為上旋、下旋和平擊，根據握拍手可分為單手反拍和雙手反拍。單手反拍就是僅僅用持球手反手擊球，控制的範圍大，便於偽裝放小球，但穩定性較雙反差。雙手反拍是利用另一隻手輔助持拍

In [79]:
 #approach 2: turn vectorstore into retriever
 retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
 # TODO: explain arguments, explain runnable passthrough
 retrieved_docs = retriever.invoke("決勝盤會在哪一種網球比賽中使用？")
 print(retrieved_docs[0].page_content)

  0%|          | 0/1 [00:00<?, ?it/s]

大多數比賽由多盤組成，而且總盤數一般為單數。大滿貫男子單打比賽有5盤，率先贏得3盤的選手獲勝；而其他比賽一般有3盤，率先贏得2盤的選手獲勝。發球權在每局比賽後交換，不受一盤比賽的開始或結束影響。決勝局亦算作一局比賽。一場比賽的總比分可以只給出總盤數的比分，也可列出所有局數的比分，但必須先列出勝者的比分。例如7-56-76-47-6採用的就是後一種記分方式，表示獲勝的一方贏得了第一、第三和第四盤的勝利，而第二和第四盤是靠決勝局決出勝負的。在括號中的數字表示的是輸的一方在決勝局中得到的分數，例如第二盤決勝局的比分就是4-7，而第四盤決勝局的比分則是8-6。在現在網球比賽中，為了加快比賽進程，在雙方平局局數足夠高時將不再延長比賽而採用一局特殊的決勝局結束一盤比賽。決勝局規則為現在大多數公開賽所採用，但並不是強制使用，如溫布頓網球錦標賽在第五盤時仍然採用長盤勝負。在一般的決勝局中，首先獲得7分且淨勝2分以上的選手贏得該盤比賽。由於在決勝局中雙方交替發球，無論哪一方獲勝該局均不會被記入保發/破發局。在雙打比賽中，有時也會使用一種特殊的決勝盤。如果雙方在前兩盤各下一盤，那麼第三盤將為一局決勝負，首先贏得10分且淨勝2分以上的一方贏得比賽。


In [91]:
from tqdm.notebook import tqdm
results = []
retriever.vectorstore.embeddings.show_progress_bar=False
for _, record in tqdm(clean_test.iterrows(), total=len(clean_test)):
  q = record['question']
  retrieved_docs = vectorstore.similarity_search_with_score(q, k=5)
  results.append(retrieved_docs)


  0%|          | 0/800 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [92]:
top1, top5 = 0,0
for i, result in enumerate(results):
  record = clean_test.iloc[i]
  gt = record['id']
  top1_pred = result[0][0].metadata['id']
  if top1_pred == gt:
    top1+=1
  for (doc, score) in result:
    pred = doc.metadata['id']
    if pred == gt:
      top5+=1
print(top5/len(results))
print(top1/len(results))

0.9090909090909091
0.2727272727272727


### RAG: integrating LLM to generate


### <font color="red">Ignore</font>. Build a custom LLM (API call or lcal LLM)

In [ ]:
# doc ref: https://python.langchain.com/v0.1/docs/modules/model_io/llms/custom_llm/

In [93]:
# pull RAG prompts from hub
# prompt usage API doc: https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/
from langchain import hub


# TODO: custom prompt, explain what are prompts object
prompt = hub.pull("rlm/rag-prompt")
print(prompt.messages)
example_messages = prompt.invoke(
    {"context": "filler context1", "question": "filler question1"}
)
print(example_messages.to_string())
print(example_messages.to_messages())

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]
Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question1 
Context: filler context1 
Answer:
[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question1 \nContex

## <font color=red> TODO: explain "chain" in detail </font>

In [95]:

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
# chat openai https://api.python.langchain.com/en/latest/chat_models/langchain_openai.chat_models.azure.AzureChatOpenAI.html
llm = AzureChatOpenAI(
    openai_api_version='2024-05-01-preview',
    azure_deployment='gpt-4o',
    azure_endpoint='https://mtknycuragwebinar.openai.azure.com',
    openai_api_key=userdata.get('mtk-open-key'),
)

# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     # | StrOutputParser()
# )
# for chunk in rag_chain.stream("What is Task Decomposition?"):
#     print(chunk, end="", flush=True)

llm.invoke(example_messages)

AIMessage(content="I don't know.", response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 67, 'total_tokens': 71}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-ecc2f9ec-8f32-460f-84de-080f06522f9a-0', usage_metadata={'input_tokens': 67, 'output_tokens': 4, 'total_tokens': 71})

In [97]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d3dca92cb54a4e03d8031ba3a2207dd4916404d9fd50eca2d6d05d0418539df7
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [106]:
import evaluate

rouge = evaluate.load('rouge') ## import the rouge scorer

for r, record in clean_test.iterrows():
  q = record['question']
  retrieved_docs = vectorstore.similarity_search_with_score(q, k=5)
  gt = record['answer']
  gt_doc_message = prompt.invoke(
    {"context": record['context'], "question": q}
  )
  ref_response = llm.invoke(gt_doc_message)
  rag_message = prompt.invoke(
    {"context": retrieved_docs[0][0].page_content, "question": q}
  )
  rag_response = llm.invoke(rag_message)
  print(gt)
  print(ref_response)
  print(rag_response)
  break

梵語
content='夜柔吠陀與阿闥婆吠陀均可以作為研究梵語的參考。' response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 459, 'total_tokens': 486}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-d7c68eee-5b2c-416c-90db-88c1c406fa73-0' usage_metadata={'input_tokens': 459, 'output_tokens': 27, 'total_tokens': 486}
content='夜柔吠陀與阿闥婆吠陀均可以作為研究梵語的參考。' response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 459, 'total_tokens': 486}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-e9d620ef-c860-4d7c-9a3e-6b51aa373275-0' usage_metadata={'input_tokens': 459, 'output_tokens': 27, 'total_tokens': 486}


In [109]:
print(ref_response.content)
print(gt)

夜柔吠陀與阿闥婆吠陀均可以作為研究梵語的參考。
梵語


In [110]:
rouge.compute(predictions=[ref_response.content],
              references=[gt],
)

{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}